In [ ]:
import numpy as np
import pickle
from learnable_crf import LearnableCrf
from lib import *
from scipy.special import expit as sigmoid

In [ ]:
r = 0
with open('cache/df_train.{}.pickle'.format(r), mode='rb') as h:
    df = pickle.load(h)
leaves = np.nonzero([x[0] in x[1] for x in zip(df['label'], df['pseudo_label'])])[0]
Y_train = df['label'][leaves]
with open('cache/df_val_test.pickle', mode='rb') as h:
    df_val, df_test = pickle.load(h)
Y_val = df_val['label']
Y_test = df_test['label']
with open('cache/hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)
state_space = hex_data['state_space']
# state_space = filter(lambda x: x[:20].any(), state_space)

In [ ]:
model_Phi_train = np.load('results/caffe_baseline/iter_Phi_train.{}.npy'.format(r))[:, leaves]
model_Phi_val = np.load('results/caffe_baseline/iter_Phi_val.{}.npy'.format(r))
model_Phi_test = np.load('results/caffe_baseline/iter_Phi_test.{}.npy'.format(r))
M = len(model_Phi_train)

In [ ]:
[get_accuracy(model_Phi_val[i], Y_val, lim_states=False) for i in range(0, M)]

In [ ]:
[get_accuracy(model_Phi_test[i], Y_test, lim_states=False) for i in range(0, M)]

In [ ]:
lcrf = [LearnableCrf(model_Phi_train[i], Y_train) for i in range(0, M)]
[get_accuracy(lcrf[i].predict(model_Phi_val[i]), Y_val) for i in range(0, M)]

In [ ]:
[get_accuracy(lcrf[i].predict(model_Phi_test[i]), Y_test) for i in range(0, M)]